# Demo for HiPSter Inference using Illustris TNG

## Generate the HiPS tiles

The HiPS tiles are generated by reconstructing the data with the HiPS position on the spherical
latent space.

<img src="./images/HEALPix.png" width="50%">

Source: [@Fernique_2015]

In [ ]:
import hipster

hipster.HiPSGenerator(
    decoder=hipster.Inference("data/illustris/models/decoder.onnx"),
    image_maker=hipster.ImageMaker()
    max_order=2,
    hierarchy=4,
    hips_path="output/hips-illustris",
).execute()

## Catalog of training dataset



In [ ]:
hipster.VOTableGenerator(
    encoder=hipster.Inference("data/gaia/models/gaia-vae-8-nll-normal-v0/encoder.onnx"),
    data_directory="data/gaia/parquet",
    output_file="output/gaia.vot",
).execute()

## Visualize HiPS tiles and catalog using Aladin

- [Aladin-Lite](https://github.com/cds-astro/aladin-lite)
- [PyAladin](https://github.com/cds-astro/ipyaladin)

In [ ]:
from ipyaladin import Aladin
from ipywidgets import Layout

aladin = Aladin(
    survey="output/hips-gaia-model",
    height=600,
    fov=180,
    layout=Layout(width="50%"),
    show_fullscreen_control=False,
)
aladin.add_catalog_from_URL("output/gaia.vot", {"source_size": 5, "color": "red"})
aladin